# Generate multimodal travel time matrix (ttm)

This script is to generate a multimodal ttm for accessibility calculation in _r5_.<br/>
Modes: walking, biking, transit, car

It takes inspiration from https://github.com/rafapereirabr/otp-travel-time-matrix/blob/master/python_script.py

## 1. Variables definition and data import

In [1]:
# definitions
import sys
import numpy as np
import pandas as pd
import geopandas as gpd
import datetime as dt
import tracc
from r5py import TransportNetwork, TravelTimeMatrixComputer, TransitMode, LegMode
from datetime import datetime,date,timedelta
import matplotlib.pyplot as plt
sys.argv.append(["--max-memory", "8G"])


data_folder = "/Users/azanchetta/OneDrive - The Alan Turing Institute/demoland_data"


# regional level files: (require previous editing)
oas_centroids_file = f"{data_folder}/processed/OA_centroids_TyneWear.gpkg" # used for population origin
oas_file = f"{data_folder}/processed/authorities/OA_TyneWear.gpkg" # needed for visualisation purposes
region_lads_file = f"{data_folder}/processed/authorities/LADs_tynewear.shp" # needed in order to filter greenspace data within the regional boundaries
workingplacezones_centroids_file = f"{data_folder}/processed/authorities/WPZ_centroids_tynewear.gpkg"

# national level files
greenspace_file = f"{data_folder}/raw/accessibility/OS Open Greenspace (GPKG) GB/data/opgrsp_gb.gpkg"
osm_data_file = f"{data_folder}/raw/accessibility/tyne-and-wear-latest.osm.pbf"
gtfs_data_file = f"{data_folder}/raw/accessibility/itm_north_east_gtfs.zip"

In [3]:
# import

# origins (IE output areas, OAs)
oas_centroids = gpd.read_file(oas_centroids_file,
                              layer="OA_centroids_TyneWear")
oas_centroids['id'] = oas_centroids['OA11CD'] # Origin dataset must contain an 'id' column
oas_centroids_wgs84 = oas_centroids.to_crs("epsg:4326")
oas_centroids.head()

# destination data
# green space sites' entrances
gs_entrances = gpd.read_file(greenspace_file,
                        layer = "AccessPoint")

gs_entrances.head()
# WPZ centroids
wpz_centroids = gpd.read_file(workingplacezones_centroids_file,
                              layer = "WPZ_centroids_tynewear")
wpz_centroids.head()
wpz_centroids['id'] = wpz_centroids['wz11cd'] # Destination dataset must contain an 'id' column

# network data
# uploaded in the sequent operation

In [4]:
# for testing purposes
# selecting from origin and destinations only a few points
# IE selecting first 10 rows
origins = oas_centroids.loc[:99, :]
destinations = wpz_centroids.loc[:99, :]
# origins = oas_centroids
# destinations = wpz_centroids

In [5]:
origins.head()
origins.crs #epsg:27700
origins = origins.to_crs("epsg:4326")
origins.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
origins.explore()

In [6]:
destinations.head()
destinations.crs #epsg:27700
destinations = destinations.to_crs("epsg:4326")
destinations.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [ ]:
destinations.explore()

## 2. ttm generation and playing around

In [7]:
### load in transport network

transport_network = TransportNetwork(
    osm_data_file,
    [
        gtfs_data_file
    ]
)

In [7]:
# example of ttm generation
# ttm_walking_OAtoGS = TravelTimeMatrixComputer(
#     transport_network,
#     origins=orig,
#     destinations=dest,
#     max_time=dt.timedelta(seconds=900), # restricting travel to 15min
#     speed_walking=4.8,
#     transport_modes=[LegMode.WALK]
# )
# ttm_walking_OAtoGS = ttm_walking_OAtoGS.compute_travel_times()
# ttm_walking_OAtoGS.head()

ttm_transit_comp = TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    departure=dt.datetime(2023,1,19,8,30),
    transport_modes=[TransitMode.TRANSIT,
                     LegMode.WALK]
)
ttm_transit = ttm_transit_comp.compute_travel_times()
ttm_transit.head()

# destinations
# gs: entrances + oas centroids
# jobs: wpz centroids + oas centroids

/usr/local/anaconda3/envs/demoland_r5/lib/python3.9/site-packages/r5py/r5/regional_task.py:224: RuntimeWarning: Departure time 2023-01-19 08:30:00 is outside of the time range covered by currently loaded GTFS data sets.
  warnings.warn(


,from_id,to_id,travel_time
0,E00041377,E33000251,78.0
1,E00041377,E33000799,18.0
2,E00041377,E33000257,43.0
3,E00041377,E33000079,33.0
4,E00041377,E33000174,55.0


In [82]:
from itertools import product
# generate dataframe with all from_id and all to_id pairs
prod = product(origins['id'].unique(),
               destinations['id'].unique())
empty_ttm = pd.DataFrame(prod)
empty_ttm.columns= ['from_id', 'to_id']
empty_ttm.head()

,from_id,to_id
0,E00041377,E33000251
1,E00041377,E33000799
2,E00041377,E33000257
3,E00041377,E33000079
4,E00041377,E33000174


In [ ]:
# # Create a default request for a given departure time
# req = otp.createRequest()
# req.setDateTime(2015, 9, 15, 10, 00, 00)  # set departure time
# req.setMaxTimeSec(7200)                   # set a limit to maximum travel time (seconds)
# req.setModes('WALK,BUS,RAIL')             # define transport mode
# req.setClampInitialWait(0)                # clamp the initial wait time to zero
# # req.maxWalkDistance = 3000                 # set the maximum distance (in meters) the user is willing to walk
# # req.walkSpeed = walkSpeed                 # set average walking speed ( meters ?)
# # req.bikeSpeed = bikeSpeed                 # set average cycling speed (miles per hour ?)
# # ?ERROR req.setSearchRadiusM(500)                 # set max snapping distance to connect trip origin to street network

# # Read Points of Destination - The file points.csv contains the columns GEOID, X and Y.
# points = otp.loadCSVPopulation('points.csv', 'Y', 'X')
# dests = otp.loadCSVPopulation('points.csv', 'Y', 'X')


# # Create a CSV output
# matrixCsv = otp.createCSVOutput()
# matrixCsv.setHeader([ 'origin', 'destination', 'walk_distance', 'travel_time', 'boardings' ])

# # Start Loop
# for origin in points:
#   print "Processing origin: ", origin
#   req.setOrigin(origin)
#   spt = router.plan(req)
#   if spt is None: continue

#   # Evaluate the SPT for all points
#   result = spt.eval(dests)
  
#   # Add a new row of result in the CSV output
#   for r in result:
#     matrixCsv.addRow([ origin.getStringData('GEOID'), r.getIndividual().getStringData('GEOID'), r.getWalkDistance() , r.getTime(),  r.getBoardings() ])

# # Save the result
# matrixCsv.save('traveltime_matrix.csv')

# # Stop timing the code

In [8]:
trial_comp = TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    departure=dt.datetime(2023,1,19,8,30),
    # max_time = dt.timedelta(seconds=900),
    transport_modes=[LegMode.CAR]
)
trial = trial_comp.compute_travel_times()
trial.travel_time.isna().sum()

0

In [9]:
trial.head()

,from_id,to_id,travel_time
0,E00041377,E33000251,17
1,E00041377,E33000799,7
2,E00041377,E33000257,16
3,E00041377,E33000079,12
4,E00041377,E33000174,16


In [88]:
# defining variables
date_time = '2023,01,19,9,30'
max_time = dt.timedelta(seconds=900)
walking_speed = 4.8
cycling_speed = 16
modes_lut = pd.DataFrame([
                            # ['TRANSIT', TransitMode.TRANSIT, LegMode.WALK],
                          ['CAR', '', LegMode.CAR],
                          ['BICYCLE', '', LegMode.BICYCLE],
                        #   ['WALK','', LegMode.WALK]],
                        ],
                         columns=('Mode', 'Transit_mode', 'Leg_mode'))
modes_lut.head()

def list_making(s,z): # function to generate custom list of transit+transport more for the parameter transport_modes in TravelTimeMatrixComputer
    return [s] + [z]

# df.lookup(df.index, df['best'])

for row in modes_lut.itertuples():
    mode = row.Mode
    transit_mode = row.Transit_mode
    leg_mode = row.Leg_mode
    transport_mode = list_making(transit_mode,leg_mode) # creating list of objects for transport_modes parameter
    print('bbb', transport_mode)

    print('The current mode is:', mode, 'transit is', transit_mode, 'transport var is', transport_mode)
    traveltimematrix = TravelTimeMatrixComputer(
        transport_network,
        origins=origins,
        destinations=destinations,
        departure=dt.datetime.strptime(date_time, '%Y,%m,%d,%H,%M'),
        speed_walking = walking_speed,
        speed_cycling = cycling_speed,
        transport_modes = transport_mode
    )

    ttm = traveltimematrix.compute_travel_times()
    ttm.isna().sum() # checking the ttm actually calculated something
    empty_ttm = empty_ttm.merge(ttm,
                    how='outer',
                    left_on=['from_id','to_id'],
                    right_on = ['from_id','to_id'])
    
    print('finished calculating ttm for mode', mode)



bbb ['', <LegMode.CAR: <java object 'com.conveyal.r5.api.util.LegMode'>>]
The current mode is: CAR transit is  transport var is ['', <LegMode.CAR: <java object 'com.conveyal.r5.api.util.LegMode'>>]
finished calculating ttm for mode CAR
bbb ['', <LegMode.BICYCLE: <java object 'com.conveyal.r5.api.util.LegMode'>>]
The current mode is: BICYCLE transit is  transport var is ['', <LegMode.BICYCLE: <java object 'com.conveyal.r5.api.util.LegMode'>>]
finished calculating ttm for mode BICYCLE


In [80]:
ttm.head()

,from_id,to_id,travel_time
0,E00041377,E33000251,68.0
1,E00041377,E33000799,9.0
2,E00041377,E33000257,50.0
3,E00041377,E33000079,29.0
4,E00041377,E33000174,54.0


In [68]:
# example of ttm generation
ttm_walk = TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    # max_time=dt.timedelta(seconds=900), # restricting travel to 15min
    speed_walking=4.8,
    transport_modes=[LegMode.WALK]
)
ttm_walk = ttm_walk.compute_travel_times()
ttm_walk.head()

,from_id,to_id,travel_time
0,E00041377,E33000251,NaN
1,E00041377,E33000799,26.0
2,E00041377,E33000257,116.0
3,E00041377,E33000079,77.0
4,E00041377,E33000174,NaN


In [81]:
# example of ttm generation
ttm_bike = TravelTimeMatrixComputer(
    transport_network,
    origins=origins,
    destinations=destinations,
    # max_time=dt.timedelta(seconds=900), # restricting travel to 15min
    speed_walking=4.8,
    speed_cycling=16,
    transport_modes=[LegMode.BICYCLE]
)
ttm_bike = ttm_bike.compute_travel_times()
ttm_bike.head()

,from_id,to_id,travel_time
0,E00041377,E33000251,68.0
1,E00041377,E33000799,9.0
2,E00041377,E33000257,50.0
3,E00041377,E33000079,29.0
4,E00041377,E33000174,54.0


In [ ]:
a=list(str(12345))
for q in a:
    print(chr(int(q)))

In [53]:
class Student:
    def __init__(self, name, age):
        self.name = name
        self.age = age


studentList = []
studentList.append(Student("Alex", 20))
studentList.append(Student("Bob", 21))
studentList.append(Student("Ira", 15))

for student in studentList:
    print('Name : {}, Age : {}'.format(student.name,student.age))

Name : Alex, Age : 20
Name : Bob, Age : 21
Name : Ira, Age : 15
